In [1]:
# !pip install torch --quiet
# !pip uninstall torchtext -y
# !pip install torchtext==0.10.0 --quiet

In [2]:
from torchtext.datasets import AG_NEWS

In [3]:
# !pip install torchdata
train_iter = AG_NEWS(split='train')

train.csv: 29.5MB [00:00, 67.6MB/s]


In [4]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [5]:
for (line_number, (label, line)) in enumerate(train_iter):
  print(label, line)
  if line_number == 19:
    break

3 Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
3 Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
3 Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
3 Oil prices soar to all-time record, posing new menace to US economy (AFP) AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections.


In [6]:
from torch.utils.data import DataLoader
train_iter = AG_NEWS(split = 'train')

In [7]:
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False)

In [8]:
next(iter(dataloader))

[tensor([3, 3, 3, 3, 3, 3, 3, 3]),
 ("Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
  'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.',
  "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.",
  'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.',
  'Oil prices soar to all-time re

In [9]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [10]:
help(get_tokenizer)

Help on function get_tokenizer in module torchtext.data.utils:

get_tokenizer(tokenizer, language='en')
    Generate tokenizer function for a string sentence.
    
    Args:
        tokenizer: the name of tokenizer function. If None, it returns split()
            function, which splits the string sentence by space.
            If basic_english, it returns _basic_english_normalize() function,
            which normalize the string first and split by space. If a callable
            function, it will return the function. If a tokenizer library
            (e.g. spacy, moses, toktok, revtok, subword), it returns the
            corresponding library.
        language: Default en
    
    Examples:
        >>> import torchtext
        >>> from torchtext.data import get_tokenizer
        >>> tokenizer = get_tokenizer("basic_english")
        >>> tokens = tokenizer("You can now install TorchText using pip!")
        >>> tokens
        >>> ['you', 'can', 'now', 'install', 'torchtext', 'using

In [11]:
tokenizer = get_tokenizer("basic_english")

In [12]:
tokens = tokenizer("You can now install TorchText using pip!")
tokens

['you', 'can', 'now', 'install', 'torchtext', 'using', 'pip', '!']

In [13]:
help(build_vocab_from_iterator)

Help on function build_vocab_from_iterator in module torchtext.vocab:

build_vocab_from_iterator(iterator: Iterable, min_freq: int = 1, specials: Optional[List[str]] = None, special_first: bool = True) -> torchtext.vocab.Vocab
    Build a Vocab from an iterator.
    
    Args:
        iterator: Iterator used to build Vocab. Must yield list or iterator of tokens.
        min_freq: The minimum frequency needed to include a token in the vocabulary.
        specials: Special symbols to add. The order of supplied tokens will be preserved.
        special_first: Indicates whether to insert symbols at the beginning or at the end.
    
    
    Returns:
        torchtext.vocab.Vocab: A `Vocab` object
    
    Examples:
        >>> #generating vocab from text file
        >>> import io
        >>> from torchtext.vocab import build_vocab_from_iterator
        >>> def yield_tokens(file_path):
        >>>     with io.open(file_path, encoding = 'utf-8') as f:
        >>>         for line in f:
    

In [14]:
tokenizer = get_tokenizer("basic_english")
train_iter = AG_NEWS(split = 'train')

def yield_tokens(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])

In [15]:
def my_func():
  print("Hello! Welcome to the Dothraki Kingdon")
  yield
  print("Have you brought some Gold for us?")
  yield
  print("NO? Then you must die!!!")

In [16]:
temp_my_func = my_func()
next(temp_my_func)

Hello! Welcome to the Dothraki Kingdon


In [17]:
next(temp_my_func)

Have you brought some Gold for us?


In [18]:
# next(temp_my_func)

In [19]:
vocab.set_default_index(vocab["<unk>"])

In [20]:
vocab(['here', 'is', 'an', 'example', 'of', 'alien', 'invasion', 'they', 'are', 'called', 'bangalorites'])

[475, 21, 30, 5297, 6, 7838, 3042, 67, 42, 440, 0]

In [21]:
vocab["<unk>"]

0

In [22]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [23]:
# text_pipeline("Here is an exmaple of alient invasion, and they are called bangaloriters")
text_pipeline("No")

[78]

In [24]:
label_pipeline('19')

18

In [25]:
from torch.utils.data import DataLoader
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def collate_fn(batch):
  src_batch, tgt_batch = [], []
  for src_batch, tgt_batch in batch:
    src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip('\n')))
    tgt_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip('\n')))
  src_batch = pad_sequences(src_batch, padding_value=PAD_IDX)
  tgt_batch = pad_sequences(tgt_batch, padding_value=PAD_IDX)
  return src_batch, tgt_batch

In [26]:
weight = torch.randn(10, 5)
weight

tensor([[-2.6809e+00, -4.8683e-01, -1.1393e+00,  3.9214e-02,  2.0237e-01],
        [ 4.9862e-01, -1.0139e+00,  1.5631e+00, -1.4300e+00,  6.5936e-01],
        [-7.3319e-01, -5.6577e-01,  6.0029e-01, -3.6710e-01,  6.1115e-02],
        [ 7.1037e-01,  2.8070e-01,  1.2768e+00,  6.1373e-01,  1.2786e+00],
        [-4.9051e-01,  6.0958e-02, -2.0721e-01, -1.0211e+00, -1.6249e+00],
        [-5.5844e-01, -1.0487e-01, -5.2271e-01,  2.1942e-01,  1.3382e+00],
        [ 5.3808e-01, -1.1331e+00, -7.8348e-02, -1.0838e+00, -4.2695e-02],
        [ 1.7687e-03, -1.2634e+00, -1.7945e+00,  1.8705e+00,  2.5631e+00],
        [-1.4282e+00, -2.8475e-01, -4.6216e-01,  1.3882e+00,  3.9477e-02],
        [ 6.6591e-01, -1.1809e+00,  1.2942e+00, -2.4367e-01,  1.3181e-01]])

In [27]:
indices = torch.tensor([4, 1, 7])
indices

tensor([4, 1, 7])

In [28]:
embeddings = torch.nn.functional.embedding(indices, weight)
embeddings

tensor([[-4.9051e-01,  6.0958e-02, -2.0721e-01, -1.0211e+00, -1.6249e+00],
        [ 4.9862e-01, -1.0139e+00,  1.5631e+00, -1.4300e+00,  6.5936e-01],
        [ 1.7687e-03, -1.2634e+00, -1.7945e+00,  1.8705e+00,  2.5631e+00]])

In [29]:
embeddings.mean(dim=0, keepdim=True)

tensor([[ 0.0033, -0.7388, -0.1462, -0.1935,  0.5325]])

In [30]:
torch.nn.functional.embedding_bag(indices, weight, torch.tensor([0]), mode="mean")

tensor([[ 0.0033, -0.7388, -0.1462, -0.1935,  0.5325]])

In [31]:
import sys
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    # print(((batch[0][1])))  # len(batch) = 64 i.e 1 batch is a list containing 64 elements. batch[0] is of length 2 with the first one containing label and second containing sentence
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        #  print(processed_text)
        #  print(label_list)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
        #  print(offsets)
        #  print(text_list)
        #  sys.exit()
    label_list = torch.tensor(label_list, dtype=torch.int64)  
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    # print(len(text_list))
    text_list = torch.cat(text_list)  # make a long long list containing numeric rep of all sentences one after another.
    # print(len(text_list))
    # sys.exit()
    return label_list.to(device), text_list.to(device), offsets.to(device)    

In [32]:
train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [33]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        # torch.nn.functional.embedding_bag(indices, weight, torch.tensor([0]), mode="mean")
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)  # making a custom embedding with embedding of dim as "embed_dim"
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

1: World, 2: Sports, 3 Business, 4 Sci/Tech

In [34]:
train_iter = AG_NEWS(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [35]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1) # disuccees
        optimizer.step()
        # print("predicted label and label is ", predited_label, label)
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        # print("total_count and total_acc is ", total_count, total_acc )
        # sys.exit()
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [36]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

test.csv: 1.86MB [00:00, 36.7MB/s]                  


| epoch   1 |   500/ 1782 batches | accuracy    0.690
| epoch   1 |  1000/ 1782 batches | accuracy    0.856
| epoch   1 |  1500/ 1782 batches | accuracy    0.876
-----------------------------------------------------------
| end of epoch   1 | time: 18.23s | valid accuracy    0.887 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.898
| epoch   2 |  1000/ 1782 batches | accuracy    0.901
| epoch   2 |  1500/ 1782 batches | accuracy    0.904
-----------------------------------------------------------
| end of epoch   2 | time: 17.69s | valid accuracy    0.893 
-----------------------------------------------------------
| epoch   3 |   500/ 1782 batches | accuracy    0.914
| epoch   3 |  1000/ 1782 batches | accuracy    0.914
| epoch   3 |  1500/ 1782 batches | accuracy    0.913
-----------------------------------------------------------
| end of epoch   3 | time: 17.29s | valid accuracy    0.905 
-------------------------------

In [37]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.908


In [38]:
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a Sports news
